# Polygonization by rivers & railways - get raw data; filter by tag; polygonize

In this notebook, for each city, we
* load the city boundary polygon
* download and save raw OSM data (features) on rivers/railways for this polygon
* load the info which tags to keep (derived from **manual exploration** of each data set, cf. `03_naturalbarriers_explore.ipynb`)
* process the data set (dropping unwanted tags), merge, and polygonize
* save polygons as output ("fragmentation by rivers/railways/borders" in each city)

In [1]:
# import warnings
# warnings.filterwarnings("ignore")
import os
# import sys
# import argparse
# import random
# from time import time
# import numpy as np
import pandas as pd
# import pickle
import matplotlib.pyplot as plt
import geopandas as gpd
# from shapely.geometry import LineString, Point, MultiPoint, MultiLineString, Polygon
# from tqdm import tqdm
# from geopy.distance import distance, geodesic, great_circle
import osmnx as ox
# import networkx as nx
# import scipy.stats
# from scipy.stats import ks_2samp
# import sklearn
# import igraph
# from igraph import Graph

# from random import choice
# from bisect import bisect_left
# import copy

# from functools import partial
# import pyproj
# from pyproj import Geod
# from pyproj.crs import ProjectedCRS
# from pyproj.crs.coordinate_operation import AzimuthalEquidistantConversion
# from shapely.ops import transform

import yaml
# from utils import *
import shapely
# ox.__version__
import momepy
import contextily as cx
import folium

**Def tiny functions**
(plots for sanity check)

In [35]:
def plot_polygons(polygons, natbar, folder_plots, city_name):
    fig, ax = plt.subplots(1,1, figsize = (20,20))
    polygons.plot(ax=ax, color = "#F7F7F7", alpha = .5)
    polygons.boundary.plot(ax=ax, color = "black", lw = 1, linestyle = "dotted")
    natbar.plot(ax=ax, column="barrier_type", legend = True, lw = 10, alpha = .2)
    xlims = list(natbar[natbar["barrier_type"]=="city_border"].bounds[["minx", "maxx"]].values[0])
    ylims = list(natbar[natbar["barrier_type"]=="city_border"].bounds[["miny", "maxy"]].values[0])
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_axis_off()
    ax.set_title(city_name, fontsize = 20)
    plt.tight_layout()
    fig.savefig(folder_plots + f"{city_name}.png", dpi = 300, bbox_inches = "tight")
    plt.close()
    return None

In [2]:
proj_crs = "epsg:9311"

In [3]:
# create subfolders for output data
folders = [
    "../data/natural_barriers/",
    "../data/natural_barriers/raw/",
    "../data/natural_barriers/polygonized/"
    "../data/natural_barriers/plots/"
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# define folder paths
folder_osm = "../data/natural_barriers/raw/"
folder_poly = "../data/natural_barriers/polygonized/"
folder_plots = "../data/natural_barriers/plots/"

barrier_types = ["railway", "waterway"]

In [4]:
# read in and format CBSA table to loop through
cbsacode_file = '../data/cbsacode.csv'
df_cbsacodes = pd.read_csv(cbsacode_file)
df_cbsacodes = df_cbsacodes[["cbsacode", "name", "full_name", "geometry", "west", "south", "east", "north"]]
# convert text to shapely Polygon
df_cbsacodes["geometry"] = df_cbsacodes.geometry.apply(lambda x: shapely.from_wkt(x))
# convert to geodataframe
gdf_cbsacodes = gpd.GeoDataFrame(df_cbsacodes, crs = "EPSG:4326")
gdf_cbsacodes.head()

,cbsacode,name,full_name,geometry,west,south,east,north
0,12060,Atlanta,Atlanta,"POLYGON ((-85.33823 33.65312, -85.33842 33.654...",-85.386581,32.844640,-83.269196,34.617924
1,12420,Austin,Austin,"POLYGON ((-98.04989 30.62416, -98.04954 30.624...",-98.297600,29.630718,-97.024461,30.906188
2,12580,Baltimore,Baltimore,"POLYGON ((-77.16808 39.35396, -77.16831 39.354...",-77.311562,38.711099,-75.747761,39.721503
3,13820,Birmingham,Birmingham,"POLYGON ((-86.95366 33.8153, -86.95365 33.8202...",-87.421989,32.660117,-86.043865,34.260483
4,14460,Boston,Boston,"POLYGON ((-71.24734 43.27523, -71.24732 43.275...",-71.898773,41.565851,-70.322522,43.572787


In [5]:
gdf_cbsacodes

,cbsacode,name,full_name,geometry,west,south,east,north
0,12060,Atlanta,Atlanta,"POLYGON ((-85.33823 33.65312, -85.33842 33.654...",-85.386581,32.844640,-83.269196,34.617924
1,12420,Austin,Austin,"POLYGON ((-98.04989 30.62416, -98.04954 30.624...",-98.297600,29.630718,-97.024461,30.906188
2,12580,Baltimore,Baltimore,"POLYGON ((-77.16808 39.35396, -77.16831 39.354...",-77.311562,38.711099,-75.747761,39.721503
3,13820,Birmingham,Birmingham,"POLYGON ((-86.95366 33.8153, -86.95365 33.8202...",-87.421989,32.660117,-86.043865,34.260483
4,14460,Boston,Boston,"POLYGON ((-71.24734 43.27523, -71.24732 43.275...",-71.898773,41.565851,-70.322522,43.572787
5,15380,Buffalo,Buffalo,"POLYGON ((-79.01923 43.06796, -79.02076 43.068...",-79.312136,42.437997,-78.460416,43.634799
6,16740,Charlotte,Charlotte,"POLYGON ((-81.4556 35.41981, -81.4558 35.42004...",-81.537599,34.457776,-79.848479,36.058810
7,16980,Chicago,Chicago,"POLYGON ((-88.60224 41.63139, -88.61185 41.631...",-88.942146,40.736510,-86.929353,42.669906
8,17140,Cincinnati,Cincinnati,"POLYGON ((-84.87075 38.9009, -84.87123 38.9009...",-85.298623,38.473039,-83.673018,39.729263
9,17460,Cleveland,Cleveland,"POLYGON ((-81.49518 42.0964, -81.45429 42.116,...",-82.348183,40.988196,-81.002116,42.251602


Download and save data sets

In [ ]:
### Download and save OSM data

# initialize dict where we will save all data
data_dict = {}

for _, row in gdf_cbsacodes.iterrows():

    cbsacode = row.cbsacode
    city_name = row["name"]
    geom = row.geometry

    print(f"{city_name}:")
    
    data_dict[city_name] = {} 

    for barrier_type in barrier_types:
        
        barrier_path = folder_osm + f'{cbsacode}_{barrier_type}.gpkg'
        
        if os.path.exists(barrier_path):
            print(f"\t file found, loading OSM data: {barrier_type}")
            gdf = gpd.read_file(barrier_path)
        else:
            print(f"\t file not found, downloading OSM data: {barrier_type}")
            gdf = ox.features_from_polygon(
                polygon=geom,
                tags = {
                    barrier_type:True, 
                }
            )
            gdf = gdf.explode(ignore_index=True)
            gdf = gdf[["geometry",barrier_type]]
            gdf.to_file(barrier_path)
            
        data_dict[city_name][barrier_type] = gdf
        del gdf

        print("\t done")

*******
*******

# Manual verification

(cf. `03_naturalbarriers_explore.ipynb`)

*******
*******


**Load tag dict** (derived manually)

In [23]:
with open("barriertags.yml", "r") as file:
    tags = yaml.load(file, Loader=yaml.FullLoader)
tags

{'Atlanta': {'railway': ['rail', 'abandoned'], 'waterway': ['river']},
 'Austin': {'railway': ['rail'], 'waterway': ['river']},
 'Baltimore': {'railway': ['rail'], 'waterway': ['river', 'flowline']},
 'Birmingham': {'railway': ['rail'], 'waterway': ['river']},
 'Boston': {'railway': ['rail'], 'waterway': ['river', 'flowline', 'canal']},
 'Buffalo': {'railway': ['rail'], 'waterway': ['river']},
 'Charlotte': {'railway': ['rail'], 'waterway': ['river']},
 'Chicago': {'railway': ['rail'], 'waterway': ['river', 'canal']},
 'Cincinnati': {'railway': ['rail'], 'waterway': ['river']},
 'Cleveland': {'railway': ['rail'], 'waterway': ['river']},
 'Columbus': {'railway': ['rail', 'disused'], 'waterway': ['river']},
 'Dallas': {'railway': ['rail'], 'waterway': ['river']},
 'Denver': {'railway': ['rail'], 'waterway': ['river', 'canal']},
 'Detroit': {'railway': ['rail'], 'waterway': ['river', 'canal']},
 'Hartford': {'railway': ['rail'], 'waterway': ['river']},
 'Houston': {'railway': ['rail'], 'w

**reduce data sets** accordingly

In [ ]:
for city_name, tag_dict in tags.items():    

    gdfs = []

    cbsacode = gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].cbsacode.values[0]
    
    natbar_path = folder_poly + f"{cbsacode}_natbar.gpkg"
    polygon_path = folder_poly + f"{cbsacode}_polygons.gpkg"

    # do this only if not done yet
    if not (os.path.exists(natbar_path) and os.path.exists(polygon_path)):
        print("Runing for", city_name)
        city_border = gpd.GeoDataFrame(
            {
                "geometry": [gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].boundary.values[0]],
                "barrier_type": "city_border"
            },
            crs = gdf_cbsacodes.crs
        )
        
        gdfs.append(city_border)

        for barrier_type in barrier_types:
            gdf = data_dict[city_name][barrier_type].copy()
            gdf = gdf[(gdf.geom_type=="LineString")&(gdf[barrier_type]).isin(tag_dict[barrier_type])]
            gdf["barrier_type"] = gdf[barrier_type]
            del gdf[barrier_type]
            # data_dict[city_name][barrier_type] = gdf
            gdfs.append(gdf)

        # now we have a list of 3 gdfs: city border, railway, waterway. concat them all:
        # and save to file - these are the polygon outlines
        natbar = pd.concat(gdfs).reset_index(drop=True)
        natbar.to_file(folder_poly + f"{cbsacode}_natbar.gpkg", index = False)

        # and polygonize & save
        polygons = momepy.FaceArtifacts(natbar).polygons[["geometry"]].set_crs(natbar.crs)
        polygons.to_file(folder_poly + f"{cbsacode}_polygons.gpkg", index = False)

        plot_polygons(polygons, natbar, folder_plots, city_name)

        del natbar, polygons

## postprocess charlotte - remove short rivers!!

In [36]:
city_name = "Charlotte"
tag_dict = tags[city_name].copy()

gdfs = []

cbsacode = gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].cbsacode.values[0]
print(cbsacode)

natbar_path = folder_poly + f"{cbsacode}_natbar.gpkg"
polygon_path = folder_poly + f"{cbsacode}_polygons.gpkg"

# do this only if not done yet

city_border = gpd.GeoDataFrame(
    {
        "geometry": [gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].boundary.values[0]],
        "barrier_type": "city_border"
    },
    crs = gdf_cbsacodes.crs
)

gdfs.append(city_border)

for barrier_type in barrier_types:
    barrier_path = folder_osm + f'{cbsacode}_{barrier_type}.gpkg'
    gdf = gpd.read_file(barrier_path)
    gdf = gdf[(gdf.geom_type=="LineString")&(gdf[barrier_type]).isin(tag_dict[barrier_type])]
    gdf["barrier_type"] = gdf[barrier_type]
    del gdf[barrier_type]
    # for waterways, drop short pieces:
    if barrier_type == "waterway":
        print(len(gdf))
        gdf_proj = gdf.to_crs(proj_crs).copy()
        gdf_proj = gdf_proj[gdf_proj.length > 1000].copy()
        gdf = gdf_proj.to_crs(gdf.crs)
        print(len(gdf))
    gdfs.append(gdf)

# now we have a list of 3 gdfs: city border, railway, waterway. concat them all:
# and save to file - these are the polygon outlines
natbar = pd.concat(gdfs).reset_index(drop=True)
natbar.to_file(folder_poly + f"{cbsacode}_natbar.gpkg", index = False)

# and polygonize & save
polygons = momepy.FaceArtifacts(natbar).polygons[["geometry"]].set_crs(natbar.crs)
polygons.to_file(folder_poly + f"{cbsacode}_polygons.gpkg", index = False)

plot_polygons(polygons, natbar, folder_plots, city_name)

del natbar, polygons

16740
4922
195


/var/folders/66/3jkth_7d5gggg6pyr8yywwt40000gn/T/ipykernel_82931/1886862675.py:45: UserWarning: No threshold found. Either your dataset it too small or the distribution of the face artifact index does not follow the expected shape.
  polygons = momepy.FaceArtifacts(natbar).polygons[["geometry"]].set_crs(natbar.crs)


## reprocess miami - merging tiny polygons resulting from short canals

In [ ]:
city_name = "Miami"
tag_dict = tags[city_name].copy()

gdfs = []

cbsacode = gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].cbsacode.values[0]
print(cbsacode)

natbar_path = folder_poly + f"{cbsacode}_natbar.gpkg"
polygon_path = folder_poly + f"{cbsacode}_polygons.gpkg"

city_border = gpd.GeoDataFrame(
    {
        "geometry": [gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].boundary.values[0]],
        "barrier_type": "city_border"
    },
    crs = gdf_cbsacodes.crs
)

gdfs.append(city_border)

for barrier_type in barrier_types:
    gdf = data_dict[city_name][barrier_type].copy()
    gdf = gdf[(gdf.geom_type=="LineString")&(gdf[barrier_type]).isin(tag_dict[barrier_type])]
    gdf["barrier_type"] = gdf[barrier_type]
    del gdf[barrier_type]
    # # for waterways, drop short pieces:
    # if barrier_type == "waterway":
    #     print(len(gdf))
    #     gdf_proj = gdf.to_crs(proj_crs).copy()
    #     gdf_proj = gdf_proj[gdf_proj.length > 1000].copy()
    #     gdf = gdf_proj.to_crs(gdf.crs)
    #     print(len(gdf))
    data_dict[city_name][barrier_type] = gdf
    gdfs.append(gdf)

# now we have a list of 3 gdfs: city border, railway, waterway. concat them all:
# and save to file - these are the polygon outlines
natbar = pd.concat(gdfs).reset_index(drop=True)
natbar.to_file(folder_poly + f"{cbsacode}_natbar.gpkg", index = False)

# and polygonize & save
polygons = momepy.FaceArtifacts(natbar).polygons[["geometry"]].set_crs(natbar.crs)
polygons.to_file(folder_poly + f"{cbsacode}_polygons.gpkg", index = False)

plot_polygons(polygons, natbar, folder_plots, city_name)

del natbar, polygons

In [24]:
city_name = "Miami"
barrier_type = "waterway"
cbsacode = gdf_cbsacodes[gdf_cbsacodes["name"]==city_name].cbsacode.values[0]

In [30]:
# remove naively preprocessed files
os.remove(folder_poly + f"{cbsacode}_polygons.gpkg")
os.remove(folder_poly + f"{cbsacode}_natbar.gpkg")

In [25]:
folder_osm = "../data/natural_barriers/raw/"
barrier_path = folder_osm + f'{cbsacode}_{barrier_type}.gpkg'
gdf = gpd.read_file(barrier_path)
assert all([t in ["Point", "LineString", "Polygon"] for t in gdf.geom_type.unique()]), "Unexpected geom type, double check"

In [26]:
gdf = gdf[gdf.geom_type=="LineString"]
gdf = gdf[gdf[barrier_type].isin(tags[city_name][barrier_type])].copy().reset_index(drop=True)
gdf["barrier_type"] = gdf[barrier_type]
del gdf[barrier_type]

In [27]:
gdf = gdf.to_crs(proj_crs)

In [28]:
gdf["lengths"] = gdf.length

In [31]:
gdf.head()

,waterway,geometry,lengths
0,canal,"LINESTRING (1960362.497 -1942799.302, 1960370....",28897.907611
1,canal,"LINESTRING (1959247.718 -1943074.301, 1959236....",10341.118596
2,canal,"LINESTRING (1961319.713 -1911217.763, 1961885....",17191.302485
3,river,"LINESTRING (1944043.565 -1981153.407, 1944034....",806.932961
4,river,"LINESTRING (1945621.766 -1980293.249, 1945630....",328.104854


try merging small polygons to their (also small) neighbours?


In [ ]:
poly = gpd.read_file(f"../data/natural_barriers/polygonized/33100_polygons.gpkg")

* add area column
* find reasonable threshold for merging adjacent polygons (contiguous! cf simp) that are too small

In [ ]:
from esda.shape import diameter_ratio
from libpysal import graph

In [ ]:
# project
poly = poly.to_crs(proj_crs)
poly["sqkm"] = poly.area / 10**6
poly["i"] = poly.geometry.apply(lambda x: diameter_ratio(x)[0])

In [ ]:
# poly[poly.i<0.12].explore(column = "i", tiles = "cartodb positron")

In [ ]:
poly_merge = poly[(poly.i > 0.12)&(poly.sqkm<7)].copy()

In [ ]:
#poly_merge.explore(tiles="cartodb positron", column="sqkm")

In [ ]:
rook = graph.Graph.build_contiguity(poly_merge, rook=True)

In [ ]:
poly_merge["label"] = poly_merge.index

In [ ]:
poly_merge["neighbours"] = poly_merge.label.apply(lambda x: rook.neighbors[x])

In [ ]:
poly_merge["comp"] = rook.component_labels

In [ ]:
# poly_merge.explore(tiles="cartodb positron", column="comp", cmap = "Dark2")

In [ ]:
# manually untangle: 642 & 643 are their own comp; also 701

In [ ]:
last_comp = int(poly_merge.comp.max())
poly_merge.loc[[642,643],"comp"] = last_comp + 1
poly_merge.loc[701,"comp"] = last_comp + 2

In [ ]:
# use info to merge rest of polygons

In [ ]:
poly["merge"] = None

In [ ]:
for ix, row in poly_merge.iterrows():
    poly.loc[ix,"merge"] = row.comp

In [ ]:
poly.loc[poly_merge.index]

In [ ]:
# these that have no particular comp to be merged with - keep as they are
poly_keep = poly[poly["merge"].isna()].copy()

In [ ]:
# these that need to be merged - merge them
geoms = []
for comp, mygroup in poly_merge.groupby("comp"):
    geoms.append(mygroup.geometry.union_all())

In [ ]:
new_gdf = gpd.GeoDataFrame(
    {
        "geometry": list(poly_keep["geometry"]) + geoms
    }, 
    crs = poly.crs
)
new_gdf = new_gdf.to_crs("EPSG:4326")

In [ ]:
m = new_gdf.explore(tiles="cartodb positron", name = "new")
poly.explore(m=m, name = "old", color = "red")
folium.LayerControl().add_to(m)
m

redo this but without dropping indeces -- then we can track which polygons to merge!

In [ ]:
poly.explore()